In [1]:
import stim
import pymatching
import numpy as np

In [8]:
# code from https://github.com/quantumlib/Stim/blob/main/doc/getting_started.ipynb
def count_logical_errors(circuit: stim.Circuit, num_shots: int) -> int:
    # Sample the circuit.
    sampler = circuit.compile_detector_sampler()
    detection_events, observable_flips = sampler.sample(num_shots, separate_observables=True)

    # Configure a decoder using the circuit.
    detector_error_model = circuit.detector_error_model(decompose_errors=True)
    matcher = pymatching.Matching.from_detector_error_model(detector_error_model)

    # Run the decoder.
    predictions = matcher.decode_batch(detection_events)

    # Count the mistakes.
    num_errors = 0
    for shot in range(num_shots):
        actual_for_shot = observable_flips[shot]
        predicted_for_shot = predictions[shot]
        if not np.array_equal(actual_for_shot, predicted_for_shot):
            num_errors += 1
    return num_errors

circuit = stim.Circuit.generated("repetition_code:memory", rounds=1, distance=9, before_round_data_depolarization=0.15)
num_shots = 100_000
num_logical_errors = count_logical_errors(circuit, num_shots)
print("there were", num_logical_errors, "wrong predictions (logical errors) out of", num_shots, "shots")


KeyboardInterrupt



In [6]:
circuit.to_crumble_url()

'https://algassert.com/crumble#circuit=R_0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15_16;TICK;DEPOLARIZE1(0.15)0_2_4_6_8_10_12_14_16;CX_0_1_2_3_4_5_6_7_8_9_10_11_12_13_14_15;TICK;CX_2_1_4_3_6_5_8_7_10_9_12_11_14_13_16_15;TICK;MR_1_3_5_7_9_11_13_15;DT(1,0)rec[-8];DT(3,0)rec[-7];DT(5,0)rec[-6];DT(7,0)rec[-5];DT(9,0)rec[-4];DT(11,0)rec[-3];DT(13,0)rec[-2];DT(15,0)rec[-1];M_0_2_4_6_8_10_12_14_16;DT(1,1)rec[-8]_rec[-9]_rec[-17];DT(3,1)rec[-7]_rec[-8]_rec[-16];DT(5,1)rec[-6]_rec[-7]_rec[-15];DT(7,1)rec[-5]_rec[-6]_rec[-14];DT(9,1)rec[-4]_rec[-5]_rec[-13];DT(11,1)rec[-3]_rec[-4]_rec[-12];DT(13,1)rec[-2]_rec[-3]_rec[-11];DT(15,1)rec[-1]_rec[-2]_rec[-10];OI(0)rec[-1]'